In [1]:
import os
import zipfile
import random
import tensorflow as tf
import keras 


import random
import sys 

print(tf.__version__)
print(sys.version)

2.8.0
3.10.4 (main, Mar 25 2022, 00:00:00) [GCC 11.2.1 20220127 (Red Hat 11.2.1-9)]


In [2]:
import numpy as np
import matplotlib.pyplot as plt


# use seaborn plotting defaults
import seaborn as sns; sns.set()
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.image as mpimg
from PIL import Image
from PIL import ImageFilter


In [3]:
loaded = tf.saved_model.load('./custom_best') 
converter = tf.lite.TFLiteConverter.from_saved_model("./custom_best")
tflite_best_custom = "./tflite_custom_best"
with open(tflite_best_custom, "wb") as f:
          f.write(converter.convert())

2022-05-18 14:08:40.596498: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-18 14:08:40.637743: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-18 14:08:40.637999: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-18 14:08:40.638729: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [4]:
with open(tflite_best_custom, 'rb') as f:
    tflite_best_custom_instance = f.read()
    

In [5]:
interpreter_custom_best = tf.lite.Interpreter(model_content=tflite_best_custom_instance)
interpreter_custom_best.allocate_tensors()
input_index = interpreter_custom_best.get_input_details()[0]['index']
output_index = interpreter_custom_best.get_output_details()[0]['index']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [ ]:
normals = os.listdir('/bigdata3/OCT2017/all_data/NORMAL/')
drusen = os.listdir('/bigdata3/OCT2017/all_data/DRUSEN//')
dme = os.listdir('/bigdata3/OCT2017/all_data/DME/')
cnv = os.listdir('/bigdata3/OCT2017/all_data/CNV/')

In [7]:
types = ['CNV', 'DME', 'DRUSEN', 'NORMAL'] 


In [25]:
normal_image_path = os.path.join('/bigdata3/OCT2017/all_data/NORMAL/', random.choice(normals))

print(f'testing image {normal_image_path}')
normal_image = tf.io.read_file(normal_image_path)
normal_image = tf.io.decode_jpeg(normal_image, channels=3)
fni = tf.image.resize(
    normal_image,
    (256, 256),
    # method=ResizeMethod.BILINEAR,
    preserve_aspect_ratio=False,
    antialias=False,
    name=None
)
fni = np.float32(fni.numpy()/255.)
fni = fni.reshape((1, 256, 256, 3))
interpreter_custom_best.set_tensor(input_index, fni)
interpreter_custom_best.invoke()
result = interpreter_custom_best.get_tensor(output_index)
print(result)
amax = np.argmax(result)
print(amax)
percent = result[0][amax] * 100
image_type = types[amax]
print(f'type: {image_type} confidence: {percent}%')

testing image /bigdata3/OCT2017/all_data/NORMAL/NORMAL-6322340-1.jpeg
[[5.9025217e-05 2.7097864e-02 5.9611993e-03 9.6688199e-01]]
3
type: NORMAL confidence: 96.68819904327393%


In [29]:
drusen_image_path = os.path.join('/bigdata3/OCT2017/all_data/DRUSEN/', random.choice(drusen))

print(f'testing image {drusen_image_path}')
drusen_image = tf.io.read_file(drusen_image_path)
drusen_image = tf.io.decode_jpeg(drusen_image, channels=3)
fni = tf.image.resize(
    drusen_image,
    (256, 256),
    # method=ResizeMethod.BILINEAR,
    preserve_aspect_ratio=False,
    antialias=False,
    name=None
)
fni = np.float32(fni.numpy()/255.)
fni = fni.reshape((1, 256, 256, 3))
interpreter_custom_best.set_tensor(input_index, fni)
interpreter_custom_best.invoke()
result = interpreter_custom_best.get_tensor(output_index)
print(result)
amax = np.argmax(result)
print(amax)
percent = result[0][amax] * 100
image_type = types[amax]
print(f'type: {image_type} confidence: {percent}%')

testing image /bigdata3/OCT2017/all_data/DRUSEN/DRUSEN-7457346-2.jpeg
[[4.0439041e-05 1.4724962e-04 9.7607893e-01 2.3733335e-02]]
2
type: DRUSEN confidence: 97.60789275169373%
